# PCS — Symbolic First Notebook
Author: Demetrios Agourakis (ORCID 0000-0002-8596-5097)  
License: CC BY 4.0 (text) / MIT (code)  
Date: 2025-08-31

**Reproducibility**: fixed seeds; record environment + dataset versions.  
**Outputs**: write tables to `/data/processed/...` and figures to `/figures/...`.

## 01 — SWOW Loader & Graph Build

In [ ]:
# Set paths to SWOW raw files under data/raw_public/swow/<lang>/
SWOW_RAW_DIR = 'data/raw_public/swow/en/'
OUTPUT_DIR = 'data/processed/swow/en/'

import os, pandas as pd, networkx as nx
os.makedirs(OUTPUT_DIR, exist_ok=True)

# TODO: load SWOW CSVs; build directed/weighted graph (cue→association with weights)
# df = pd.read_csv(os.path.join(SWOW_RAW_DIR, 'swow_en.csv'))
# G = nx.DiGraph()
# for row in df.itertuples():
#     G.add_edge(row.cue, row.assoc, weight=row.weight)

# TODO: save graph (GraphML/edgelist) and a summary CSV
print('SWOW loader stub ready.')